In [1]:
import trimesh
import numpy as np
import pyrender
import pandas as pd
import json

## Constructing the 3D model using trimesh and pyrender

In [1]:
import pandas as pd
import numpy as np
import trimesh
import pyrender

path = "actionsequences.csv"
df = pd.read_csv(path, header=None, names=['Activity'])
df['Activity'] = df['Activity'].str.replace('["\']', '', regex=True)

def create_cylinder(length, radius):
    return trimesh.creation.cylinder(radius=radius, height=length)

def create_leg(radius, height):
    return trimesh.creation.cylinder(radius=radius, height=height)

def reconstructed(df):
    # Desk dimensions and other parameters
    desktop_length = 1.2
    desktop_width = 0.6
    desktop_thickness = 0.05
    edge_radius = 0.025
    ball_radius = 0.03
    leg_radius = 0.05
    leg_height = 0.7

    # Create the top surface of the desktop
    top_surface = trimesh.creation.box(extents=(desktop_length, desktop_width, desktop_thickness))
    color = (255, 255, 255, 255)  # Red color in RGBA format (R, G, B, Alpha)
    top_surface.visual.vertex_colors = np.array([color] * len(top_surface.vertices))

    # Calculate translations for edges
    edge1_translation = [0, desktop_width / 2 - edge_radius - desktop_thickness / 2 + 0.05, edge_radius- 0.016]
    edge2_translation = [0, -desktop_width / 2 + edge_radius + desktop_thickness / 2 - 0.05, edge_radius -0.016]
    edge3_translation = [desktop_length / 2 - edge_radius + 0.05, 0, desktop_thickness / 2 + edge_radius - 0.06]  ##
    edge4_translation = [-desktop_length / 2 + edge_radius - 0.05, 0, desktop_thickness / 2 + edge_radius -0.06]

    # Create edges
    edge1 = create_cylinder(desktop_length, edge_radius)
    edge1.apply_translation(edge1_translation)
    edge1.apply_transform(trimesh.transformations.rotation_matrix(np.pi / 2, [0, 1, 0]))

    edge2 = create_cylinder(desktop_length, edge_radius)
    edge2.apply_translation(edge2_translation)
    edge2.apply_transform(trimesh.transformations.rotation_matrix(np.pi / 2, [0, 1, 0]))

    edge3 = create_cylinder(desktop_width+0.03, edge_radius)
    edge3.apply_translation(edge3_translation)
    edge3.apply_transform(trimesh.transformations.rotation_matrix(np.pi / 2, [1, 0, 0]))

    edge4 = create_cylinder(desktop_width+0.03, edge_radius)
    edge4.apply_translation(edge4_translation)
    edge4.apply_transform(trimesh.transformations.rotation_matrix(np.pi / 2, [1, 0, 0]))

    # Create balls at the corners
    ball1_translation = [desktop_length / 2 - ball_radius+0.06, desktop_width / 2 - ball_radius+0.02, desktop_thickness / 2]
    ball2_translation = [desktop_length / 2 - ball_radius+0.06, -desktop_width / 2 + ball_radius-0.02, desktop_thickness / 2]
    ball3_translation = [-desktop_length / 2 + ball_radius-0.06, desktop_width / 2 - ball_radius+0.02, desktop_thickness / 2]
    ball4_translation = [-desktop_length / 2 + ball_radius-0.06, -desktop_width / 2 + ball_radius-0.02, desktop_thickness / 2]

    ball1 = trimesh.creation.icosphere(radius=ball_radius)
    ball1.apply_translation(ball1_translation)

    ball2 = trimesh.creation.icosphere(radius=ball_radius)
    ball2.apply_translation(ball2_translation)

    ball3 = trimesh.creation.icosphere(radius=ball_radius)
    ball3.apply_translation(ball3_translation)

    ball4 = trimesh.creation.icosphere(radius=ball_radius)
    ball4.apply_translation(ball4_translation)

    # Create the legs using the provided create_leg function
    leg1 = create_leg(leg_radius, leg_height)
    leg1.apply_translation([desktop_length / 2 - leg_radius, desktop_width / 2 - leg_radius, -desktop_thickness - 0.3])

    leg2 = create_leg(leg_radius, leg_height)
    leg2.apply_translation([desktop_length / 2 - leg_radius, -desktop_width / 2 + leg_radius, -desktop_thickness - 0.3])

    leg3 = create_leg(leg_radius, leg_height)
    leg3.apply_translation([-desktop_length / 2 + leg_radius, desktop_width / 2 - leg_radius, -desktop_thickness - 0.3])

    leg4 = create_leg(leg_radius, leg_height)
    leg4.apply_translation([-desktop_length / 2 + leg_radius, -desktop_width / 2 + leg_radius, -desktop_thickness - 0.3])

    
    desktop = top_surface
    ihlr, ie, ihsr, ivsr = 1, 1, 1, 1

    for index, row in df.iterrows():
        activity = row['Activity']

        if activity == 'inserting horizontal long rods':
            if ihlr == 1:
                desktop += edge1
                ihlr += 1
            elif ihlr == 2:
                desktop += edge2
                ihlr += 1

        elif activity == 'inserting elbows':
            if ie == 1:
                desktop += ball1
                ie += 1
            elif ie == 2:
                desktop += ball2
                ie += 1
            elif ie == 3:
                desktop += ball3
                ie += 1
            elif ie == 4:
                desktop += ball4
                ie += 1

        elif activity == 'inserting horizontal short rods':
            if ihsr == 1:
                desktop += edge3
                ihsr += 1
            elif ihsr == 2:
                desktop += edge4
                ihsr += 1

        elif activity == 'inserting vertical short rods':
            if ivsr == 1:
                desktop += leg1
                ivsr += 1
            elif ivsr == 2:
                desktop += leg2
                ivsr += 1
            elif ivsr == 3:
                desktop += leg3
                ivsr += 1
            elif ivsr == 4:
                desktop += leg4
                ivsr += 1


        mesh = pyrender.Mesh.from_trimesh(desktop, smooth=False)

        scene = pyrender.Scene()
        scene.add(mesh)


        viewer = pyrender.Viewer(scene, use_raymond_lighting=True)


reconstructed(df)


## merging two videos 

In [2]:
import cv2
import os

def add_text_to_frame(frame, text, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, color=(0, 0, 255), thickness=2):
    """
    Adds text to a given frame.

    Args:
    - frame (np.ndarray): The image frame to which text is added.
    - text (str): The text to be added.
    - font (int): Font type.
    - font_scale (float): Font scale factor.
    - color (tuple): Text color.
    - thickness (int): Thickness of the text.
    """
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    text_x = (frame.shape[1] - text_size[0]) // 2
    
    text_y = (frame.shape[0] // 10) + text_size[1]  
    position = (text_x, text_y)
    cv2.putText(frame, text, position, font, font_scale, color, thickness, lineType=cv2.LINE_AA)

def create_video_from_frames(image_folder, output_video_path, frame_rate=30):
    
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    images.sort() 

    if not images:
        print("No images found in the provided folder.")
        return

    print(f"Found {len(images)} images in {image_folder}")

    
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    if frame is None:
        print(f"Failed to read the first image {images[0]}")
        return

    height, width, layers = frame.shape
    frame_size = (width, height)

    print(f"Frame size: {frame_size}")

    
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  
    video = cv2.VideoWriter(output_video_path, fourcc, frame_rate, frame_size)

    if not video.isOpened():
        print(f"Failed to open video writer for {output_video_path}")
        return

    # Define text for each set of frames
    #texts = [
     #   "Inserting the horizontal long rods",  # Frames 1 & 2
     #   "Inserting the horizontal long rods",  # Frames 1 & 2
    #  "Inserting elbows",  # Frames 3, 4, 5 & 6
     #   "Inserting elbows",
     #   "Inserting elbows",
    upsample_factors = [
        108, 108,  # Frames 1 & 2
        98, 98, 98, 98,  # Frames 3, 4, 5 & 6
        218, 218,  # Frames 7 & 8
        130, 130, 130, 130  # Frames 9, 10, 11 & 12
    ]

    for i, image in enumerate(images):
        image_path = os.path.join(image_folder, image)
        frame = cv2.imread(image_path)
        if frame is None:
            print(f"Failed to read image {image}")
            continue

        

        print(f"Writing frame {image} with shape {frame.shape}")
        for _ in range(upsample_factors[i]):
            video.write(frame)

    video.release()
    print(f"Video saved as {output_video_path}")


image_folder = '/home/student/Downloads/Screenshot_Sunday_Evening_Perferctly_Alligned' 
output_video_path = '/home/student/Downloads/output_video.avi'  


frame_rate = 24 


create_video_from_frames(image_folder, output_video_path, frame_rate)


Found 12 images in /home/student/Downloads/Screenshot_Sunday_Evening_Perferctly_Alligned
Frame size: (1610, 1025)
Writing frame 01.png with shape (1025, 1610, 3)
Writing frame 02.png with shape (1025, 1610, 3)
Writing frame 03.png with shape (1025, 1610, 3)
Writing frame 04.png with shape (1025, 1610, 3)
Writing frame 05.png with shape (1025, 1610, 3)
Writing frame 06.png with shape (1025, 1610, 3)
Writing frame 07.png with shape (1025, 1610, 3)
Writing frame 08.png with shape (1025, 1610, 3)
Writing frame 09.png with shape (1025, 1610, 3)
Writing frame 10.png with shape (1025, 1610, 3)
Writing frame 11.png with shape (1025, 1610, 3)
Writing frame 12.png with shape (1025, 1610, 3)
Video saved as /home/student/Downloads/output_video.avi


In [ ]:
from moviepy.editor import VideoFileClip, clips_array


video1_path = '/home/student/Downloads/output_video.avi'
video2_path = '/home/student/Downloads/output_video_seq_3_view_5_nm_v1.mp4'  
output_path = '/home/student/Downloads/side_by_side_video.avi'  


clip1 = VideoFileClip(video1_path)
clip2 = VideoFileClip(video2_path)


clip2 = clip2.resize(height=clip1.h)


final_clip = clips_array([[clip1, clip2]])


final_clip.write_videofile(output_path, codec='png', fps=clip1.fps)

print(f"Video saved as {output_path}")
